<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Chat Completion

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.14.1"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.SemanticKernel, 1.14.1

Using Azure OpenAI Service


### Step 2 - Execute Chat Completion with Prompt Execution Settings

> "If you can’t describe what you are doing as a process, you don’t know what you’re doing." 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide a list of decision frameworks that can help improve the quality of decisions.
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2500, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

Console.WriteLine(chatResult.Content);

Certainly! Here’s a list of decision frameworks that can help improve the quality of decisions:

1. **SWOT Analysis**:
   - Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.

2. **Decision Matrix**:
   - A tool that helps prioritize options based on specific criteria, allowing for a systematic comparison.

3. **Cost-Benefit Analysis**:
   - Compares the costs and benefits of different options to determine the most economically viable choice.

4. **Pareto Analysis (80/20 Rule)**:
   - Focuses on identifying the most impactful factors (20%) that will yield the majority (80%) of results.

5. **Multi-Criteria Decision Analysis (MCDA)**:
   - Evaluates multiple conflicting criteria in decision-making, often using weighted scoring systems.

6. **The Eisenhower Matrix**:
   - Helps prioritize tasks based on urgency and importance, aiding in decision-making about where to focus efforts.

7. **OODA Loop (Observe, Orient, Decide, Act)**:
   - A decisio

### Step 3 - Execute a Semantic Kernel Chat Completion with Streaming

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [4]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt))
{
    Console.Write(content);
}

Certainly! Here are several decision frameworks that can help improve the quality of decisions:

1. **SWOT Analysis**: Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.

2. **DECIDE Model**: A structured approach that stands for Define the problem, Establish criteria, Consider alternatives, Identify the best alternative, Develop and implement a plan, and Evaluate the decision.

3. **Cost-Benefit Analysis**: Compares the costs and benefits of different options to determine which offers the best return on investment.

4. **Decision Tree**: A graphical representation that outlines possible consequences of different choices, helping visualize paths and outcomes.

5. **Analytic Hierarchy Process (AHP)**: Breaks down complex decisions into simpler components, allowing for pairwise comparisons to prioritize options based on defined criteria.

6. **Pareto Analysis**: Focuses on identifying the most impactful factors by emphasizing the 80/20 rule, whe

### Step 4 - Execute a Semantic Kernel with Chat History

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt
* User Message - A message from the user
* Assistante Message - A message from the LLM

This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [5]:
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
// Simple prompt to list 5 decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five decision frameworks that can help improve the quality of decisions.
""";

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Certainly! Here are five decision frameworks that can help improve the quality of decisions:

### 1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Overview**: SWOT analysis helps identify internal and external factors that can impact the decision.
   - **Steps**:
     1. List the strengths of the option or decision’s internal characteristics.
     2. List weaknesses that hinder the option internally.
     3. Identify external opportunities that can be leveraged.
     4. Highlight external threats that could pose risks.
   - **Application**: Useful for strategic planning, business decisions, and personal growth.

### 2. **Decision Matrix Analysis**
   - **Overview**: Also known as the Weighted Scoring Model, this helps compare multiple options against a set of criteria.
   - **Steps**:
     1. List the possible options.
     2. Determine criteria important for the decision.
     3. Assign weights to each criterion based on its importance.
     4. Score each op

### Step 5 - Using Semantic Kernel Chat History in Conversation

With Semantic Kernel maintaining state with the ChatHistory object, a user can interact with the LLM in a natural way. Note the interaction below, where the user is asking which of the particular decision frameworks are best suited to purchase a car. Without the ChatHistory being passed along, the LLM would not have had the proper context to answer the question.

> "Price is what you pay. Value is what you get."  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

In [6]:
// Simple prompt that asks a follow-up question about the purchase of a car
// Note: No reference is made to what previous frameworks were listed
var simpleDecisionPromptFollowup = """
Which of these 5 decision frameworks would work best for making a decision to purchase a car. 
Select only the single best one.
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowup);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

For making a decision to purchase a car, the **Decision Matrix** (also known as the Prioritization Matrix) would be the most effective decision framework.

### Decision Matrix for Car Purchase
#### Steps
1. **List Options**: Identify all the car models you are considering.
2. **Determine Criteria**: Identify key factors that are important for your decision, such as:
   - Price
   - Fuel efficiency
   - Maintenance cost
   - Safety features
   - Brand reputation
   - Resale value
   - Comfort and interior features
3. **Score Options**: Rate each car based on how well it meets each criterion, typically on a scale (e.g., 1 to 10).
4. **Weight Criteria**: Assign weights to each criterion based on how relevant or important each factor is to you (e.g., safety might be more important than minor interior features).
5. **Calculate Totals**: Multiply the scores by the weights and sum them up for each car option.

This framework offers the following advantages for a car purchase:
- **Comprehensiv

> "Take time to deliberate, but when the time for action comes, stop thinking and go in."
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [7]:
var simpleDecisionPromptFollowupPartTwo = """
Which of these 5 decision frameworks listed above are used by the military intelligence community?
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupPartTwo);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

The **OODA Loop** is the decision framework from the list that is most commonly associated with and used by the military intelligence community. 

### The OODA Loop
#### Overview:
- **Developed By**: Colonel John Boyd, a United States Air Force military strategist.
- **Purpose**: To make rapid and effective decisions in combat and other dynamic environments by iterating through a cycle of Observing, Orienting, Deciding, and Acting.

#### Steps:
1. **Observe**: Gather current data and information about the environment, including enemy actions, terrain, and other critical factors.
2. **Orient**: Analyze the gathered data to generate situational awareness, taking into account the strategic context, cultural influences, and experience.
3. **Decide**: Based on the analysis, select the best course of action from the available options.
4. **Act**: Implement the decision and immediately put the plan into action. Evaluate the results and feedback to inform the next cycle.

#### Rationale for Mi

### Step 6 - Semantic Kernel inspecting Chat History 

The Semantic Kernel Chat History is a transparent object that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

In [8]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistory.Count());

foreach (var message in chatHistory)
{
    Console.WriteLine($"# {message.Role}: {message.Content}");
}

Number of chat interactions: 7
# system: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
# user: Provide five decision frameworks that can help improve the quality of decisions.
# assistant: Certainly! Decision-making frameworks can greatly enhance the quality and efficiency of decisions by offering structured approaches. Here are five decision frameworks:

### 1. SWOT Analysis
**Purpose**: Evaluate the Strengths, Weaknesses, Opportunities, and Threats related to a decision or a project.
**Steps**:
- **Strengths**: Identify internal attributes that are beneficial.
- **Weaknesses**: Identify internal attributes that are harmful.
- **Opportunities**: Identify external conditions that are beneficial.
- **Threats**: Identify external conditions that could cause trouble.
**Application**: Use this analysis 